# 8 Puzzle solver
* Parsa KamaliPour - 97149081
* In this repository we're going to solve this puzzle using $ A^* $ and $ IDA $

#### imports:

In [1]:
import pandas as pd
import numpy as np

#### Test case 1:

In [28]:
input_puzzle = [
    [1, 2, 3],
    [4, 0, 5],
    [7, 8, 6]
]

print('Input: ')
print(pd.DataFrame(input_puzzle))
print()

desired_output = [
    [1, 2, 3],
    [4, 5, 0],
    [7, 8, 6]
]

print('Desired Output:')
print(pd.DataFrame(desired_output))

Input: 
   0  1  2
0  1  2  3
1  4  0  5
2  7  8  6

Desired Output:
   0  1  2
0  1  2  3
1  4  5  0
2  7  8  6


### code's configs:

In [14]:
heuristic_method = input("Enter the desired Heuristic method: h1 or h2")
f_function_omega = input("Enter the desired f function omega: 2 is Greedy, "
                         "0 is Uninformed best-first search, "
                         "0 < omega <= 1 is A*")


### Matrix to dictionary converter

In [ ]:
class Mat2dict:

    def __init__(self, matrix):
        self.matrix = matrix
        self.dic = {}
        self.convert()


    def convert(self):
        for r in range(len(self.matrix)):
            for c in range(len(self.matrix[0])):
                self.dic[self.matrix[r][c]] = (r, c)

        return self.dic


### the heuristic calculator class:

* H1 heuristic (misplaced tiles)

    $ \Sigma_{i=1}^{9} \; if \; currentPuzzleBoard[node_i] \; != \; goalPuzzleBoard[node_i]$
  
    $ then \; h(state_y) = h(state_y) + 1$

* H2 heuristic (manhattan distance)

    $ goalPuzzleBoard.find(currentPuzzleBoard[node_i]) $
    
    $ retrieve \; Row \; \& \; Col \; of \; goal $
  
    $ manhattanDistance = |(goal.row - current.row)| + |(goal.col - current.col)| $
    
    $ TotalHeuristic[state_i] = \Sigma_{i=1}^{9} manhattanDistance_i $

In [29]:
class Heuristic:

    def __init__(self, node, current_puzzle, desired_answer, method):
        self.method = method
        self.node = node
        self.current_puzzle = current_puzzle
        self.desired_answer = desired_answer
        self.current_puzzle_dict = Mat2dict(self.current_puzzle)
        self.desired_answer_dict = Mat2dict(self.desired_answer)

        if method == 'h1':
            self.h1_misplaced_tiles()
        elif method == 'h2':
            self.h2_manhattan_distance()

    def h1_misplaced_tiles(self):
        misplaced_counter = 0
        for row in range(len(self.current_puzzle)):
            for col in range(len(self.current_puzzle[0])):
                if self.current_puzzle[row][col] != self.desired_answer[row][col]:
                    misplaced_counter += 1
        return misplaced_counter

    def h2_manhattan_distance(self):
        total_distance_counter = 0
        for row in range(len(self.current_puzzle)):
            for col in range(len(self.current_puzzle[0])):
                correct_row, correct_col = self.desired_answer_dict[self.current_puzzle[row][col]]
                total_distance_counter += abs(row - correct_row) + abs(col - correct_col)
        return total_distance_counter

### The node class:

In [15]:
class Node:

    def __init__(self, parent, current_puzzle, g_cost, row, col, val):

        self.current_puzzle = current_puzzle
        self.parent = parent
        self.g_cost = g_cost
        self.row = row
        self.col = col
        self.val = val
        self.h_cost = Heuristic(self, current_puzzle, desired_output, heuristic_method)
        self.f_function = (2 - f_function_omega) * self.g_cost + f_function_omega * self.h_cost

